In [78]:
import MySQLdb
import sshtunnel
import os
import numpy as np
import pandas as pd
from dotenv import load_dotenv

In [114]:
load_dotenv()
DATA_DIR = './static/data'

In [154]:
class Database:
    def __init__(self, ssh_address, ssh_username, ssh_password, remote_address, 
                 database_user, database_password, database_name):
        self._ssh_address = ssh_address
        self._ssh_username = ssh_username
        self._ssh_password = ssh_password
        self._remote_address = remote_address
        self._database_user = database_user
        self._database_password = database_password
        self._database_name = database_name
        self.tables = self._get_tables()
        
    def get_connection(self, tunnel):
        """Sets up a connection to the MySQL database"""
        connection = MySQLdb.connect(
            user = self._database_user,
            passwd = self._database_password,
            host = "127.0.0.1",
            port = tunnel.local_bind_port,
            db = self._database_name
        )
        
        return connection
    
    def get_tunnel(self):
        """Sets up and returns an SSH tunnel"""
        sshtunnel.SSH_TIMEOUT = 10.0
        sshtunnel.TUNNEL_TIMEOUT = 10.0
        tunnel = sshtunnel.SSHTunnelForwarder(
            ssh_address_or_host = (self._ssh_address),
            ssh_username = self._ssh_username,
            ssh_password = self._ssh_password,
            remote_bind_address = (self._remote_address, 3306)
        )
        
        return tunnel
    
    def _get_tables(self):
        """Returns a list of all tables in the database by querying the database"""
        with self.get_tunnel() as tunnel:
            connection = self.get_connection(tunnel)
            cursor = connection.cursor()
            cursor.execute("SHOW TABLES;")
            results = cursor.fetchall()
            
        results = [result[0] for result in results]
        return results
    
    def get_tables(self):
        """Returns a list of all tables in the database by referencing the object"""
        return self.tables
    
    def is_table(self, tableName):
        if len(self.tables) == 0:
            self.tables = self.get_tables()

        return tableName in self.tables
    
    def create_table(self, tableName, tableCols, dataTypes, primaryKey=None):
        """Creates a new table in the database"""
        assert not self.is_table(tableName), f"{tableName} already exists."            
    
        command = "CREATE TABLE"
        command += f" {tableName} ( "
        for col, dtype in zip(tableCols, dataTypes):
            command += f'`{col}` {dtype}, '
            
        if primaryKey is not None:
            command += "PRIMARY KEY ("
            for key in primaryKey:
                command += f"`{key}`, "
                
        command = command[:-2]
        command += ')'
        
        print(command)
        with self.get_tunnel() as tunnel:
            connection = self.get_connection(tunnel)
            cursor = connection.cursor()
            cursor.execute(command)
            connection.commit()
            
        # add the table name into the list of tables within the object for increased speed
        self.tables += [f"{tableName}"]
            
    def describe_table(self, tableName):
        """Returns a description of the table"""
        assert self.is_table(tableName), f"{tableName} does not exist."
        
        with self.get_tunnel() as tunnel:
            connection = self.get_connection(tunnel)
            cursor = connection.cursor()
            cursor.execute(f"DESCRIBE {tableName};")
            results = cursor.fetchall()
            
        results = pd.DataFrame(data=results, columns=['Field', 'Type', 'Null', 'Key', 'Default', 'Extra'])
        return results
    
    def drop_table(self, tableName):
        """Drops a table form the database"""
        assert self.is_table(tableName), f"{tableName} does not exist."
        
        with self.get_tunnel() as tunnel:
            connection = self.get_connection(tunnel)
            cursor = connection.cursor()
            cursor.execute(f"DROP TABLE {tableName}")
            connection.commit()
            
    def insert_values(self, tableName, values, columns=None):
        assert isinstance(values, list), "values must be a list of tuples"
        assert len(values) > 0, "values cannot be empty"
        numCols = len(values[0])
        for row in values:
            assert isinstance(row, tuple), "values must be a list of tuples"
            assert len(row) == numCols, "all tuples must be the same number of values"
            
        if columns is not None:
            assert isinstance(columns, list), "columns must be a list of strings"
            assert all([isinstance(col, str) for col in columns]), "columns must be a list of strings"
            assert len(columns) == numCols, f"number of columns provided ({len(columns)}) does not match number of columns in provided data ({numCols})"
        
        command = f"INSERT INTO {tableName} "
        if columns is not None:
            command += "("
            for column in columns:
                command += f"`{column}`, "
            command = command[:-2] #replace comma with closing parenthesis
            command += ') '
        command += "VALUES "
        for row in values:
            command += str(row)+', '
        command = command[:-2]
        command += ';'
    
        with self.get_tunnel() as tunnel:
            connection = self.get_connection(tunnel)
            cursor = connection.cursor()
            cursor.execute(command)
            connection.commit()

In [155]:
SSH_ADDRESS = os.getenv('SSH_ADDRESS')
SSH_USER = os.getenv('SSH_USER')
SSH_PASS = os.getenv('SSH_PASS')
REMOTE_ADDRESS = os.getenv('REMOTE_ADDRESS')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')
DB_NAME = os.getenv('DB_NAME')

database = Database(SSH_ADDRESS, SSH_USER, SSH_PASS, REMOTE_ADDRESS, DB_USER, DB_PASS, DB_NAME)

In [116]:
items_df = pd.read_pickle(f'{DATA_DIR}/items_DF.pkl')
data_df = pd.read_pickle(f'{DATA_DIR}/data_DF.pkl')
items_tww = pd.read_pickle(f'{DATA_DIR}/items_TWW.pkl')
data_tww = pd.read_pickle(f'{DATA_DIR}/data_TWW.pkl')

In [156]:
dtype_conversion = {
    'int64':'INT',
    'int32':'INT',
    'string':'TINYTEXT',
    'object':'TINYTEXT',
    'O':'TINYTEXT',
    'float64':'DOUBLE',
    'float32':'DOUBLE',
    'bool':'BOOL'
}

if not database.is_table('items_df'):
    dtypes = [dtype_conversion.get(str(dtype)) for dtype in items_df.dtypes]
    database.create_table('items_df', list(items_df.columns), dataTypes=dtypes)
    
if not database.is_table('items_tww'):
    dtypes = [dtype_conversion.get(str(dtype)) for dtype in items_tww.dtypes]
    database.create_table('items_tww', list(items_tww.columns), dataTypes=dtypes)

if not database.is_table('data_df'):
    dtypes = [dtype_conversion.get(str(dtype)) for dtype in data_df.dtypes]
    database.create_table('data_df', list(data_df.columns), dataTypes=dtypes)
    
if not database.is_table('data_tww'):
    dtypes = [dtype_conversion.get(str(dtype)) for dtype in data_tww.dtypes]
    database.create_table('data_tww', list(data_tww.columns), dataTypes=dtypes)

CREATE TABLE data_df ( `profession` TINYTEXT, `character` TINYTEXT, `id` INT, `item` TINYTEXT, `reagents` TINYTEXT, `tag` TINYTEXT, `rarity` TINYTEXT, `difficulty` DOUBLE, `skill1` DOUBLE, `hsv` DOUBLE, `inspiration_skill` DOUBLE, `inspiration_percent` DOUBLE, `base_quantity` TINYTEXT, `multicraft_percent` DOUBLE, `has_reagent_qualities` BOOL, `has_embellishment_slot` BOOL, `has_missive_slot` BOOL, `has_tinker_slot` BOOL, `has_crest_slot` BOOL, `icon` TINYTEXT, `skill2` DOUBLE, `skill3` DOUBLE, `difficulty_tinker` DOUBLE, `difficulty_missive` DOUBLE, `difficulty_embellishment` DOUBLE, `difficulty_tinker_missive` DOUBLE, `difficulty_missive_embellishment` DOUBLE, `difficulty_whelpling` DOUBLE, `difficulty_whelpling_tinker` DOUBLE, `difficulty_whelpling_missive` DOUBLE, `difficulty_whelpling_embellishment` DOUBLE, `difficulty_whelpling_tinker_missive` DOUBLE, `difficulty_whelpling_missive_embellishment` DOUBLE, `difficulty_wyrm` DOUBLE, `difficulty_wyrm_tinker` DOUBLE, `difficulty_wyrm_m

In [140]:
data_df.columns

Index(['profession', 'character', 'id', 'item', 'reagents', 'tag', 'rarity',
       'difficulty', 'skill1', 'hsv',
       ...
       'odds_aspect_embellishment2', 'odds_aspect_embellishment_hsv2',
       'odds_aspect_embellishment3', 'odds_aspect_embellishment_hsv3',
       'odds_aspect_missive_embellishment1',
       'odds_aspect_missive_embellishment_hsv1',
       'odds_aspect_missive_embellishment2',
       'odds_aspect_missive_embellishment_hsv2',
       'odds_aspect_missive_embellishment3',
       'odds_aspect_missive_embellishment_hsv3'],
      dtype='object', length=189)

In [159]:
data_df[data_df.duplicated(['id', 'item'])]

,profession,character,id,item,reagents,tag,rarity,difficulty,skill1,hsv,...,odds_aspect_embellishment2,odds_aspect_embellishment_hsv2,odds_aspect_embellishment3,odds_aspect_embellishment_hsv3,odds_aspect_missive_embellishment1,odds_aspect_missive_embellishment_hsv1,odds_aspect_missive_embellishment2,odds_aspect_missive_embellishment_hsv2,odds_aspect_missive_embellishment3,odds_aspect_missive_embellishment_hsv3
118,Blacksmithing,Zarastannil,191529,Illustrious Insight,{190456: 50},Crafting Reagent,rare,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215,Cooking,Trillithia,197794,Grand Banquet of the Kalu'ak,"{200061: 2, 197788: 4, 197782: 6, 197787: 6, 1...",Consumable,rare,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216,Cooking,Trillithia,197794,Grand Banquet of the Kalu'ak,"{199345: 2, 197789: 4, 197783: 6, 197786: 6, 1...",Consumable,rare,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
291,Enchanting,Linidel,204697,Enchanted Aspect's Dreaming Crest,"{194123: 80, 194124: 15, 200113: 4, 204463: 2,...",Crafting Reagent,epic,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
293,Enchanting,Linidel,204697,Enchanted Aspect's Shadowflame Crest,"{194123: 80, 194124: 15, 200113: 4, 204463: 2,...",Crafting Reagent,epic,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299,Enchanting,Linidel,204681,Enchanted Whelpling's Dreaming Crest,"{194123: 30, 194124: 5, 200113: 1, 208395: 1}",Crafting Reagent,epic,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
301,Enchanting,Linidel,204681,Enchanted Whelpling's Shadowflame Crest,"{194123: 30, 194124: 5, 200113: 1, 204193: 1}",Crafting Reagent,epic,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
295,Enchanting,Linidel,204682,Enchanted Wyrm's Dreaming Crest,"{194123: 60, 194124: 10, 200113: 2, 204463: 1,...",Crafting Reagent,epic,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297,Enchanting,Linidel,204682,Enchanted Wyrm's Shadowflame Crest,"{194123: 60, 194124: 10, 200113: 2, 204463: 1,...",Crafting Reagent,epic,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
225,Enchanting,Linidel,191529,Illustrious Insight,{190456: 50},Crafting Reagent,rare,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
